In [ ]:
# Using Object-Oriented Programming to Analyze CMAQ Output Data
# Created by Alex Dolwick
# Adapted from code written by Dannys Ayala Terrones
# November 2022

In [1]:
# cell 1 
# Getting Packages
from __future__ import print_function
import Nio, Ngl
import os, math, sys
import numpy as np
import xarray as xr
import mpl_scatter_density
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
class Extract:

        
    '''
        __INIT__ FUNCTION:
        
        -- Step 1 in this class of functions.
        -- Loads in netcdf datasets and organizes them into a single dataset.
        -- data1 = a .nc file with 24 hours of output data. The last 20 hours of data are kept to shift from GMT to EDT.
        -- data2 = a .nc file with 24 hours of output data. The first 4 hours of data are kept to shift from GMT to EDT.
        -- MCIP1 = coordinates file corresponding to data1 file
        -- MCIP2 = coordinates file corresponding to data2 file
        -- species = which pollutant do you want to extract?
    '''
    
    def __init__(self, data1, data2, MCIP1, MCIP2, species):
        self.ds = xr.open_dataset(data1)
        self.ds_MCIP = xr.open_dataset(MCIP1)
        
        self.var_spec = self.ds[species][4:,0,:,:]
        self.lat2d = self.ds_MCIP['LAT'][0,0,:,:]   
        self.lon2d = self.ds_MCIP['LON'][0,0,:,:] 

        print('var_spec:{}'.format(self.var_spec.shape))
        print('lat2d.shape:{}'.format(self.lat2d.shape))
        print('lon2d.shape:{}'.format(self.lon2d.shape))

        self.ds_2 = xr.open_dataset(data2)
        self.ds_MCIP_2 = xr.open_dataset(MCIP2)

        self.var2_spec = self.ds_2[species][:4,0,:,:]
        self.lat2d_2 = self.ds_MCIP_2['LAT'][0,0,:,:]   
        self.lon2d_2 = self.ds_MCIP_2['LON'][0,0,:,:] 

        print('var2_spec:{}'.format(self.var2_spec.shape))
        print('lat2d_2.shape:{}'.format(self.lat2d_2.shape))
        print('lon2d_2.shape:{}'.format(self.lon2d_2.shape))

        print("Total grid cells (for reference) =", 299*459)
        
        self.spec_D1 = self.var_spec.values
        self.lat2d_Values = self.lat2d.values
        self.lon2d_Values = self.lon2d.values

        self.spec_D2 = self.var2_spec.values
        self.lat2d_2_Values = self.lat2d_2.values
        self.lon2d_2_Values = self.lon2d_2.values
        
        self.spec_CV = np.concatenate((self.spec_D1, self.spec_D2))
        print('spec_CV.shape:{}'.format(self.spec_CV.shape))
        
        print("Matching Latitude =",(self.lat2d_Values == self.lat2d_2_Values).all())
        print("Matching Longitude =",(self.lon2d_Values == self.lon2d_2_Values).all())
        
    '''
        ARRAYIFY FUNCTION:
        
        -- Step 2.
        -- Organizes dataset from Step 1 into an xarray.
    '''
    
    def arrayify(self):
        self.CMAQ_Chem_da = xr.Dataset(
            data_vars=dict(
                spec = (["x", "y", "z"], self.spec_CV),
            ),
            coords=dict(
                lon=(["y", "z"], self.lon2d_Values),
                lat=(["y", "z"], self.lat2d_Values),
                ),
            attrs=dict(
                description="Species Concentrations",
                units="Parts per Million (ppm)",
            ),
        )
        
        self.County_CMAQ_Chem = xr.where(self.CMAQ_Chem_da.lat > 34.297882  , self.CMAQ_Chem_da, np.nan)
        self.County_CMAQ_Chem = xr.where(self.County_CMAQ_Chem.lat < 34.95478058 , self.County_CMAQ_Chem , np.nan)
        self.County_CMAQ_Chem = xr.where(self.County_CMAQ_Chem.lon < -78.80546570 , self.County_CMAQ_Chem , np.nan)
        self.County_CMAQ_Chem = xr.where(self.County_CMAQ_Chem.lon > -79.46142578 , self.County_CMAQ_Chem , np.nan)
        
        self.County_CMAQ_ChemArr = self.County_CMAQ_Chem.to_array()
        
    '''
        COUNTOBS FUNCTION:
        
        -- Step 3.
        -- Counts the number of non-NA observations in the data.
        -- This function could almost certainly be made more efficient and less time-intensive.
    '''
    
    def countobs(self):
        self.count_T0 = 0 
        self.count_T1 = 0
        self.count_T2 = 0
        self.count_T3 = 0
        self.count_T4 = 0
        self.count_T5 = 0
        self.count_T6 = 0
        self.count_T7 = 0
        self.count_T8 = 0
        self.count_T9 = 0
        self.count_T10 = 0
        self.count_T11 = 0
        self.count_T12 = 0
        self.count_T13 = 0
        self.count_T14 = 0
        self.count_T15 = 0
        self.count_T16 = 0
        self.count_T17 = 0
        self.count_T18 = 0
        self.count_T19 = 0
        self.count_T20 = 0
        self.count_T21 = 0
        self.count_T22 = 0
        self.count_T23 = 0

        self.col = 299
        self.row = 459
        self.time = 24
        for i in range(self.col):
            for j in range(self.row):
                for k in range(self.time):
                    if k == 0:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T0 = self.count_T0
                        else:
                            self.count_T0 = self.count_T0 + 1
                    if k == 1:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T1 = self.count_T1
                        else:
                            self.count_T1 = self.count_T1 + 1
                    if k == 2:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T2 = self.count_T2
                        else:
                            self.count_T2 = self.count_T2 + 1
                    if k == 3:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T3 = self.count_T3
                        else:
                            self.count_T3 = self.count_T3 + 1
                    if k == 4:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T4 = self.count_T4
                        else:
                            self.count_T4 = self.count_T4 + 1
                    if k == 5:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T5 = self.count_T5
                        else:
                            self.count_T5 = self.count_T5 + 1
                    if k == 6:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T6 = self.count_T6
                        else:
                            self.count_T6 = self.count_T6 + 1
                    if k == 7:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T7 = self.count_T7
                        else:
                            self.count_T7 = self.count_T7 + 1
                    if k == 8:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T8 = self.count_T8
                        else:
                            self.count_T8 = self.count_T8 + 1
                    if k == 9:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T9 = self.count_T9
                        else:
                            self.count_T9 = self.count_T9 + 1
                    if k == 10:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T10 = self.count_T10
                        else:
                            self.count_T10 = self.count_T10 + 1
                    if k == 11:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T11 = self.count_T11
                        else:
                            self.count_T11 = self.count_T11 + 1
                    if k == 12:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T12 = self.count_T12
                        else:
                            self.count_T12 = self.count_T12 + 1
                    if k == 13:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T13 = self.count_T13
                        else:
                            self.count_T13 = self.count_T13 + 1
                    if k == 14:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T14 = self.count_T14
                        else:
                            self.count_T14 = self.count_T14 + 1
                    if k == 15:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T15 = self.count_T15
                        else:
                            self.count_T15 = self.count_T15 + 1
                    if k == 16:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T16 = self.count_T16
                        else:
                            self.count_T16 = self.count_T16 + 1
                    if k == 17:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T17 = self.count_T17
                        else:
                            self.count_T17 = self.count_T17 + 1
                    if k == 18:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T18 = self.count_T18
                        else:
                            self.count_T18 = self.count_T18 + 1
                    if k == 19:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T19 = self.count_T19
                        else:
                            self.count_T19 = self.count_T19 + 1
                    if k == 20:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T20 = self.count_T20
                        else:
                            self.count_T20 = self.count_T20 + 1
                    if k == 21:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T21 = self.count_T21
                        else:
                            self.count_T21 = self.count_T21 + 1
                    if k == 22:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T22 = self.count_T22
                        else:
                            self.count_T22 = self.count_T22 + 1
                    if k == 23:
                        if np.isnan(self.County_CMAQ_ChemArr[0,i,j,k]):
                            self.count_T23 = self.count_T23
                        else:
                            self.count_T23 = self.count_T23 + 1




         ## if there are zero, check the dimensions of the box for lat and lon            
        print(self.count_T0)
        print(self.count_T1)
        print(self.count_T2)
        print(self.count_T3)
        print(self.count_T4)
        print(self.count_T5)
        print(self.count_T6)
        print(self.count_T7)
        print(self.count_T8)
        print(self.count_T9)
        print(self.count_T10)
        print(self.count_T11)
        print(self.count_T12)
        print(self.count_T13)
        print(self.count_T14)
        print(self.count_T15)
        print(self.count_T16)
        print(self.count_T17)
        print(self.count_T18)
        print(self.count_T19)
        print(self.count_T20)
        print(self.count_T21)
        print(self.count_T22)
        print(self.count_T23)
        
    '''
        ARRAYCREATE FUNCTION:
        
        -- Step 4.
        -- Creates xarray of valid observations from Step 3.
        -- The second half of this function could almost certainly be made more efficient and less time-intensive.
    '''
    
    def arraycreate(self):
        # Latitude
        #D0
        self.output0_Lat = xr.DataArray(np.arange(self.count_T0, dtype=np.double))
        self.output0_Lat = xr.zeros_like(self.output0_Lat)
        #D1
        self.output1_Lat = xr.DataArray(np.arange(self.count_T1, dtype=np.double))
        self.output1_Lat = xr.zeros_like(self.output1_Lat)
        #D2
        self.output2_Lat = xr.DataArray(np.arange(self.count_T2, dtype=np.double))
        self.output2_Lat = xr.zeros_like(self.output2_Lat)
        #D3
        self.output3_Lat = xr.DataArray(np.arange(self.count_T3, dtype=np.double))
        self.output3_Lat = xr.zeros_like(self.output3_Lat)
        #D4
        self.output4_Lat = xr.DataArray(np.arange(self.count_T4, dtype=np.double))
        self.output4_Lat = xr.zeros_like(self.output4_Lat)
        #D5
        self.output5_Lat = xr.DataArray(np.arange(self.count_T5, dtype=np.double))
        self.output5_Lat = xr.zeros_like(self.output5_Lat)
        #D6
        self.output6_Lat = xr.DataArray(np.arange(self.count_T6, dtype=np.double))
        self.output6_Lat = xr.zeros_like(self.output6_Lat)
        #D7
        self.output7_Lat = xr.DataArray(np.arange(self.count_T7, dtype=np.double))
        self.output7_Lat = xr.zeros_like(self.output7_Lat)
        #D8
        self.output8_Lat = xr.DataArray(np.arange(self.count_T8, dtype=np.double))
        self.output8_Lat = xr.zeros_like(self.output8_Lat)
        #D9
        self.output9_Lat = xr.DataArray(np.arange(self.count_T9, dtype=np.double))
        self.output9_Lat = xr.zeros_like(self.output9_Lat)
        #D10
        self.output10_Lat = xr.DataArray(np.arange(self.count_T10, dtype=np.double))
        self.output10_Lat = xr.zeros_like(self.output10_Lat)
        #D11
        self.output11_Lat = xr.DataArray(np.arange(self.count_T11, dtype=np.double))
        self.output11_Lat = xr.zeros_like(self.output11_Lat)
        #D12
        self.output12_Lat = xr.DataArray(np.arange(self.count_T12, dtype=np.double))
        self.output12_Lat = xr.zeros_like(self.output12_Lat)
        #D13
        self.output13_Lat = xr.DataArray(np.arange(self.count_T13, dtype=np.double))
        self.output13_Lat = xr.zeros_like(self.output13_Lat)
        #D14
        self.output14_Lat = xr.DataArray(np.arange(self.count_T14, dtype=np.double))
        self.output14_Lat = xr.zeros_like(self.output14_Lat)
        #D15
        self.output15_Lat = xr.DataArray(np.arange(self.count_T15, dtype=np.double))
        self.output15_Lat = xr.zeros_like(self.output15_Lat)
        #D16
        self.output16_Lat = xr.DataArray(np.arange(self.count_T16, dtype=np.double))
        self.output16_Lat = xr.zeros_like(self.output16_Lat)
        #D17
        self.output17_Lat = xr.DataArray(np.arange(self.count_T17, dtype=np.double))
        self.output17_Lat = xr.zeros_like(self.output17_Lat)
        #D18
        self.output18_Lat = xr.DataArray(np.arange(self.count_T18, dtype=np.double))
        self.output18_Lat = xr.zeros_like(self.output18_Lat)
        #D19
        self.output19_Lat = xr.DataArray(np.arange(self.count_T19, dtype=np.double))
        self.output19_Lat = xr.zeros_like(self.output19_Lat)
        #D20
        self.output20_Lat = xr.DataArray(np.arange(self.count_T20, dtype=np.double))
        self.output20_Lat = xr.zeros_like(self.output20_Lat)
        #D21
        self.output21_Lat = xr.DataArray(np.arange(self.count_T21, dtype=np.double))
        self.output21_Lat = xr.zeros_like(self.output21_Lat)
        #D22
        self.output22_Lat = xr.DataArray(np.arange(self.count_T22, dtype=np.double))
        self.output22_Lat = xr.zeros_like(self.output22_Lat)
        #D23
        self.output23_Lat = xr.DataArray(np.arange(self.count_T23, dtype=np.double))
        self.output23_Lat = xr.zeros_like(self.output23_Lat)

        # Longitude
        #D0
        self.output0_Lon = xr.DataArray(np.arange(self.count_T0, dtype=np.double))
        self.output0_Lon = xr.DataArray(xr.zeros_like(self.output0_Lon))
        #D1
        self.output1_Lon = xr.DataArray(np.arange(self.count_T1, dtype=np.double))
        self.output1_Lon = xr.DataArray(xr.zeros_like(self.output1_Lon))
        #D2
        self.output2_Lon = xr.DataArray(np.arange(self.count_T2, dtype=np.double))
        self.output2_Lon = xr.DataArray(xr.zeros_like(self.output2_Lon))
        #D3
        self.output3_Lon = xr.DataArray(np.arange(self.count_T3, dtype=np.double))
        self.output3_Lon = xr.DataArray(xr.zeros_like(self.output3_Lon))
        #D4
        self.output4_Lon = xr.DataArray(np.arange(self.count_T4, dtype=np.double))
        self.output4_Lon = xr.DataArray(xr.zeros_like(self.output4_Lon))
        #D5
        self.output5_Lon = xr.DataArray(np.arange(self.count_T5, dtype=np.double))
        self.output5_Lon = xr.DataArray(xr.zeros_like(self.output5_Lon))
        #D6
        self.output6_Lon = xr.DataArray(np.arange(self.count_T6, dtype=np.double))
        self.output6_Lon = xr.DataArray(xr.zeros_like(self.output6_Lon))
        #D7
        self.output7_Lon = xr.DataArray(np.arange(self.count_T7, dtype=np.double))
        self.output7_Lon = xr.DataArray(xr.zeros_like(self.output7_Lon))
        #D8
        self.output8_Lon = xr.DataArray(np.arange(self.count_T8, dtype=np.double))
        self.output8_Lon = xr.DataArray(xr.zeros_like(self.output8_Lon))
        #D9
        self.output9_Lon = xr.DataArray(np.arange(self.count_T9, dtype=np.double))
        self.output9_Lon = xr.DataArray(xr.zeros_like(self.output9_Lon))
        #D10
        self.output10_Lon = xr.DataArray(np.arange(self.count_T10, dtype=np.double))
        self.output10_Lon = xr.DataArray(xr.zeros_like(self.output10_Lon))
        #D11
        self.output11_Lon = xr.DataArray(np.arange(self.count_T11, dtype=np.double))
        self.output11_Lon = xr.DataArray(xr.zeros_like(self.output11_Lon))
        #D12
        self.output12_Lon = xr.DataArray(np.arange(self.count_T12, dtype=np.double))
        self.output12_Lon = xr.DataArray(xr.zeros_like(self.output12_Lon))
        #D13
        self.output13_Lon = xr.DataArray(np.arange(self.count_T13, dtype=np.double))
        self.output13_Lon = xr.DataArray(xr.zeros_like(self.output13_Lon))
        #D14
        self.output14_Lon = xr.DataArray(np.arange(self.count_T14, dtype=np.double))
        self.output14_Lon = xr.DataArray(xr.zeros_like(self.output14_Lon))
        #D15
        self.output15_Lon = xr.DataArray(np.arange(self.count_T15, dtype=np.double))
        self.output15_Lon = xr.DataArray(xr.zeros_like(self.output15_Lon))
        #D16
        self.output16_Lon = xr.DataArray(np.arange(self.count_T16, dtype=np.double))
        self.output16_Lon = xr.DataArray(xr.zeros_like(self.output16_Lon))
        #D17
        self.output17_Lon = xr.DataArray(np.arange(self.count_T17, dtype=np.double))
        self.output17_Lon = xr.DataArray(xr.zeros_like(self.output17_Lon))
        #D18
        self.output18_Lon = xr.DataArray(np.arange(self.count_T18, dtype=np.double))
        self.output18_Lon = xr.DataArray(xr.zeros_like(self.output18_Lon))
        #D19
        self.output19_Lon = xr.DataArray(np.arange(self.count_T19, dtype=np.double))
        self.output19_Lon = xr.DataArray(xr.zeros_like(self.output19_Lon))
        #D20
        self.output20_Lon = xr.DataArray(np.arange(self.count_T20, dtype=np.double))
        self.output20_Lon = xr.DataArray(xr.zeros_like(self.output20_Lon))
        #D21
        self.output21_Lon = xr.DataArray(np.arange(self.count_T21, dtype=np.double))
        self.output21_Lon = xr.DataArray(xr.zeros_like(self.output21_Lon))
        #D22
        self.output22_Lon = xr.DataArray(np.arange(self.count_T22, dtype=np.double))
        self.output22_Lon = xr.DataArray(xr.zeros_like(self.output22_Lon))
        #D23
        self.output23_Lon = xr.DataArray(np.arange(self.count_T23, dtype=np.double))
        self.output23_Lon = xr.DataArray(xr.zeros_like(self.output23_Lon))

        # Species
        #D0
        self.output0_spec = xr.DataArray(np.arange(self.count_T0, dtype=np.double))
        self.output0_spec = xr.DataArray(xr.zeros_like(self.output0_spec))
        #D1
        self.output1_spec = xr.DataArray(np.arange(self.count_T1, dtype=np.double))
        self.output1_spec = xr.DataArray(xr.zeros_like(self.output1_spec))
        #D2
        self.output2_spec = xr.DataArray(np.arange(self.count_T2, dtype=np.double))
        self.output2_spec = xr.DataArray(xr.zeros_like(self.output2_spec))
        #D3
        self.output3_spec = xr.DataArray(np.arange(self.count_T3, dtype=np.double))
        self.output3_spec = xr.DataArray(xr.zeros_like(self.output3_spec))
        #D4
        self.output4_spec = xr.DataArray(np.arange(self.count_T4, dtype=np.double))
        self.output4_spec = xr.DataArray(xr.zeros_like(self.output4_spec))
        #D5
        self.output5_spec = xr.DataArray(np.arange(self.count_T5, dtype=np.double))
        self.output5_spec = xr.DataArray(xr.zeros_like(self.output5_spec))
        #D6
        self.output6_spec = xr.DataArray(np.arange(self.count_T6, dtype=np.double))
        self.output6_spec = xr.DataArray(xr.zeros_like(self.output6_spec))
        #D7
        self.output7_spec = xr.DataArray(np.arange(self.count_T7, dtype=np.double))
        self.output7_spec = xr.DataArray(xr.zeros_like(self.output7_spec))
        #D8
        self.output8_spec = xr.DataArray(np.arange(self.count_T8, dtype=np.double))
        self.output8_spec = xr.DataArray(xr.zeros_like(self.output8_spec))
        #D9
        self.output9_spec = xr.DataArray(np.arange(self.count_T9, dtype=np.double))
        self.output9_spec = xr.DataArray(xr.zeros_like(self.output9_spec))
        #D10
        self.output10_spec = xr.DataArray(np.arange(self.count_T10, dtype=np.double))
        self.output10_spec = xr.DataArray(xr.zeros_like(self.output10_spec))
        #D11
        self.output11_spec = xr.DataArray(np.arange(self.count_T11, dtype=np.double))
        self.output11_spec = xr.DataArray(xr.zeros_like(self.output11_spec))
        #D12
        self.output12_spec = xr.DataArray(np.arange(self.count_T12, dtype=np.double))
        self.output12_spec = xr.DataArray(xr.zeros_like(self.output12_spec))
        #D13
        self.output13_spec = xr.DataArray(np.arange(self.count_T13, dtype=np.double))
        self.output13_spec = xr.DataArray(xr.zeros_like(self.output13_spec))
        #D14
        self.output14_spec = xr.DataArray(np.arange(self.count_T14, dtype=np.double))
        self.output14_spec = xr.DataArray(xr.zeros_like(self.output14_spec))
        #D15
        self.output15_spec = xr.DataArray(np.arange(self.count_T15, dtype=np.double))
        self.output15_spec = xr.DataArray(xr.zeros_like(self.output15_spec))
        #D16
        self.output16_spec = xr.DataArray(np.arange(self.count_T16, dtype=np.double))
        self.output16_spec = xr.DataArray(xr.zeros_like(self.output16_spec))
        #D17
        self.output17_spec = xr.DataArray(np.arange(self.count_T17, dtype=np.double))
        self.output17_spec = xr.DataArray(xr.zeros_like(self.output17_spec))
        #D18
        self.output18_spec = xr.DataArray(np.arange(self.count_T18, dtype=np.double))
        self.output18_spec = xr.DataArray(xr.zeros_like(self.output18_spec))
        #D19
        self.output19_spec = xr.DataArray(np.arange(self.count_T19, dtype=np.double))
        self.output19_spec = xr.DataArray(xr.zeros_like(self.output19_spec))
        #D20
        self.output20_spec = xr.DataArray(np.arange(self.count_T20, dtype=np.double))
        self.output20_spec = xr.DataArray(xr.zeros_like(self.output20_spec))
        #D21
        self.output21_spec = xr.DataArray(np.arange(self.count_T21, dtype=np.double))
        self.output21_spec = xr.DataArray(xr.zeros_like(self.output21_spec))
        #D22
        self.output22_spec = xr.DataArray(np.arange(self.count_T22, dtype=np.double))
        self.output22_spec = xr.DataArray(xr.zeros_like(self.output22_spec))
        #D23
        self.output23_spec = xr.DataArray(np.arange(self.count_T23, dtype=np.double))
        self.output23_spec = xr.DataArray(xr.zeros_like(self.output23_spec))

        
        # getting values from extracted data and putting it into an array
        self.count_B0 = 0 
        self.count_B1 = 0
        self.count_B2 = 0
        self.count_B3 = 0
        self.count_B4 = 0
        self.count_B5 = 0
        self.count_B6 = 0
        self.count_B7 = 0
        self.count_B8 = 0
        self.count_B9 = 0
        self.count_B10 = 0
        self.count_B11 = 0
        self.count_B12 = 0
        self.count_B13 = 0
        self.count_B14 = 0
        self.count_B15 = 0
        self.count_B16 = 0
        self.count_B17 = 0
        self.count_B18 = 0
        self.count_B19 = 0
        self.count_B20 = 0
        self.count_B21 = 0
        self.count_B22 = 0
        self.count_B23 = 0

        col = 299
        row = 459
        time = 24
        for i in range(col):
            for j in range(row):
                for k in range(time):
                    if (np.isnan(self.County_CMAQ_ChemArr[0,i,j,k])==False):
                        if k == 0:
                            self.output0_Lat[self.count_B0] = self.lat2d.values[i,j]
                            self.output0_Lon[self.count_B0] = self.lon2d.values[i,j]
                            self.output0_spec[self.count_B0] = self.spec_CV[k,i,j]
                            self.count_B0 = self.count_B0 + 1
                        if k == 1:
                            self.output1_Lat[self.count_B1] = self.lat2d.values[i,j]
                            self.output1_Lon[self.count_B1] = self.lon2d.values[i,j]
                            self.output1_spec[self.count_B1] = self.spec_CV[k,i,j]
                            self.count_B1 = self.count_B1 + 1
                        if k == 2:
                            self.output2_Lat[self.count_B2] = self.lat2d.values[i,j]
                            self.output2_Lon[self.count_B2] = self.lon2d.values[i,j]
                            self.output2_spec[self.count_B2] = self.spec_CV[k,i,j]
                            self.count_B2 = self.count_B2 + 1
                        if k == 3:
                            self.output3_Lat[self.count_B3] = self.lat2d.values[i,j]
                            self.output3_Lon[self.count_B3] = self.lon2d.values[i,j]
                            self.output3_spec[self.count_B3] = self.spec_CV[k,i,j]
                            self.count_B3 = self.count_B3 + 1
                        if k == 4:
                            self.output4_Lat[self.count_B4] = self.lat2d.values[i,j]
                            self.output4_Lon[self.count_B4] = self.lon2d.values[i,j]
                            self.output4_spec[self.count_B4] = self.spec_CV[k,i,j]
                            self.count_B4 = self.count_B4 + 1
                        if k == 5:
                            self.output5_Lat[self.count_B5] = self.lat2d.values[i,j]
                            self.output5_Lon[self.count_B5] = self.lon2d.values[i,j]
                            self.output5_spec[self.count_B5] = self.spec_CV[k,i,j]
                            self.count_B5 = self.count_B5 + 1
                        if k == 6:
                            self.output6_Lat[self.count_B6] = self.lat2d.values[i,j]
                            self.output6_Lon[self.count_B6] = self.lon2d.values[i,j]
                            self.output6_spec[self.count_B6] = self.spec_CV[k,i,j]
                            self.count_B6 = self.count_B6 + 1
                        if k == 7:
                            self.output7_Lat[self.count_B7] = self.lat2d.values[i,j]
                            self.output7_Lon[self.count_B7] = self.lon2d.values[i,j]
                            self.output7_spec[self.count_B7] = self.spec_CV[k,i,j]
                            self.count_B7 = self.count_B7 + 1
                        if k == 8:
                            self.output8_Lat[self.count_B8] = self.lat2d.values[i,j]
                            self.output8_Lon[self.count_B8] = self.lon2d.values[i,j]
                            self.output8_spec[self.count_B8] = self.spec_CV[k,i,j]
                            self.count_B8 = self.count_B8 + 1
                        if k == 9:
                            self.output9_Lat[self.count_B9] = self.lat2d.values[i,j]
                            self.output9_Lon[self.count_B9] = self.lon2d.values[i,j]
                            self.output9_spec[self.count_B9] = self.spec_CV[k,i,j]
                            self.count_B9 = self.count_B9 + 1
                        if k == 10:
                            self.output10_Lat[self.count_B10] = self.lat2d.values[i,j]
                            self.output10_Lon[self.count_B10] = self.lon2d.values[i,j]
                            self.output10_spec[self.count_B10] = self.spec_CV[k,i,j]
                            self.count_B10 = self.count_B10 + 1
                        if k == 11:
                            self.output11_Lat[self.count_B11] = self.lat2d.values[i,j]
                            self.output11_Lon[self.count_B11] = self.lon2d.values[i,j]
                            self.output11_spec[self.count_B11] = self.spec_CV[k,i,j]
                            self.count_B11 = self.count_B11 + 1
                        if k == 12:
                            self.output12_Lat[self.count_B12] = self.lat2d.values[i,j]
                            self.output12_Lon[self.count_B12] = self.lon2d.values[i,j]
                            self.output12_spec[self.count_B12] = self.spec_CV[k,i,j]
                            self.count_B12 = self.count_B12 + 1
                        if k == 13:
                            self.output13_Lat[self.count_B13] = self.lat2d.values[i,j]
                            self.output13_Lon[self.count_B13] = self.lon2d.values[i,j]
                            self.output13_spec[self.count_B13] = self.spec_CV[k,i,j]
                            self.count_B13 = self.count_B13 + 1
                        if k == 14:
                            self.output14_Lat[self.count_B14] = self.lat2d.values[i,j]
                            self.output14_Lon[self.count_B14] = self.lon2d.values[i,j]
                            self.output14_spec[self.count_B14] = self.spec_CV[k,i,j]
                            self.count_B14 = self.count_B14 + 1
                        if k == 15:
                            self.output15_Lat[self.count_B15] = self.lat2d.values[i,j]
                            self.output15_Lon[self.count_B15] = self.lon2d.values[i,j]
                            self.output15_spec[self.count_B15] = self.spec_CV[k,i,j]
                            self.count_B15 = self.count_B15 + 1
                        if k == 16:
                            self.output16_Lat[self.count_B16] = self.lat2d.values[i,j]
                            self.output16_Lon[self.count_B16] = self.lon2d.values[i,j]
                            self.output16_spec[self.count_B16] = self.spec_CV[k,i,j]
                            self.count_B16 = self.count_B16 + 1
                        if k == 17:
                            self.output17_Lat[self.count_B17] = self.lat2d.values[i,j]
                            self.output17_Lon[self.count_B17] = self.lon2d.values[i,j]
                            self.output17_spec[self.count_B17] = self.spec_CV[k,i,j]
                            self.count_B17 = self.count_B17 + 1
                        if k == 18:
                            self.output18_Lat[self.count_B18] = self.lat2d.values[i,j]
                            self.output18_Lon[self.count_B18] = self.lon2d.values[i,j]
                            self.output18_spec[self.count_B18] = self.spec_CV[k,i,j]
                            self.count_B18 = self.count_B18 + 1
                        if k == 19:
                            self.output19_Lat[self.count_B19] = self.lat2d.values[i,j]
                            self.output19_Lon[self.count_B19] = self.lon2d.values[i,j]
                            self.output19_spec[self.count_B19] = self.spec_CV[k,i,j]
                            self.count_B19 = self.count_B19 + 1
                        if k == 20:
                            self.output20_Lat[self.count_B20] = self.lat2d.values[i,j]
                            self.output20_Lon[self.count_B20] = self.lon2d.values[i,j]
                            self.output20_spec[self.count_B20] = self.spec_CV[k,i,j]
                            self.count_B20 = self.count_B20 + 1
                        if k == 21:
                            self.output21_Lat[self.count_B21] = self.lat2d.values[i,j]
                            self.output21_Lon[self.count_B21] = self.lon2d.values[i,j]
                            self.output21_spec[self.count_B21] = self.spec_CV[k,i,j]
                            self.count_B21 = self.count_B21 + 1
                        if k == 22:
                            self.output22_Lat[self.count_B22] = self.lat2d.values[i,j]
                            self.output22_Lon[self.count_B22] = self.lon2d.values[i,j]
                            self.output22_spec[self.count_B22] = self.spec_CV[k,i,j]
                            self.count_B22 = self.count_B22 + 1
                        if k == 23:
                            self.output23_Lat[self.count_B23] = self.lat2d.values[i,j]
                            self.output23_Lon[self.count_B23] = self.lon2d.values[i,j]
                            self.output23_spec[self.count_B23] = self.spec_CV[k,i,j]
                            self.count_B23 = self.count_B23 + 1
                            
        print(self.output1_spec)
        
    '''
        DATASETCREATE FUNCTION:
        
        -- Step 5.
        -- Organizes xarray from Step 4.
    '''  
        
    def datasetcreate(self):
        self.CMAQ_Extract_spec = xr.Dataset(
            data_vars=dict(
                Lat = (["x"], self.output0_Lat.data),
                Lon = (["x"], self.output0_Lon.data),
                spec_T0 = (["x"], self.output0_spec.data),
                spec_T1 = (["x"], self.output1_spec.data),
                spec_T2 = (["x"], self.output2_spec.data),
                spec_T3 = (["x"], self.output3_spec.data),
                spec_T4 = (["x"], self.output4_spec.data),
                spec_T5 = (["x"], self.output5_spec.data),
                spec_T6 = (["x"], self.output6_spec.data),
                spec_T7 = (["x"], self.output7_spec.data),
                spec_T8 = (["x"], self.output8_spec.data),
                spec_T9 = (["x"], self.output9_spec.data),
                spec_T10 = (["x"], self.output10_spec.data),
                spec_T11 = (["x"], self.output11_spec.data),
                spec_T12 = (["x"], self.output12_spec.data),
                spec_T13 = (["x"], self.output13_spec.data),
                spec_T14 = (["x"], self.output14_spec.data),
                spec_T15 = (["x"], self.output15_spec.data),
                spec_T16 = (["x"], self.output16_spec.data),
                spec_T17 = (["x"], self.output17_spec.data),
                spec_T18 = (["x"], self.output18_spec.data),
                spec_T19 = (["x"], self.output19_spec.data),
                spec_T20 = (["x"], self.output20_spec.data),
                spec_T21 = (["x"], self.output21_spec.data),
                spec_T22 = (["x"], self.output22_spec.data),
                spec_T23 = (["x"], self.output23_spec.data),
            ),
            attrs=dict(
                description="Species concentrations from every hour in a 24 hour period",
                units="Parts per Million (ppm)",
                Latitude= " Extracted from 34.297882N to 34.95478058N ",
                Longitude= " Extracted from -79.46142578W to -78.80546570W "  
            ),
        )
        self.CMAQ_spec = self.CMAQ_Extract_spec.to_dataframe() 
     
    '''
        CSVCREATE FUNCTION:
        
        -- Step 6.
        -- Outputs xarray from Step 5 into a csv file.
    '''    
    
    def csvcreate(self, filename):
        self.CMAQ_spec.to_csv(filename)

In [3]:
# Locating Files
fn = "CCTM_ACONC_v53_intel17.2_soas_2016_cb6_20130714.nc"
fn_2 = "CCTM_ACONC_v53_intel17.2_soas_2016_cb6_20130715.nc"
fn_MCIP = "GRIDCRO2D_20130714.nc4"
fn_MCIP_2 = "GRIDCRO2D_20130715.nc4"

# Reading Files
ACON_file = Nio.open_file(fn, mode = "r")
ACON_file_MCIP = Nio.open_file(fn_MCIP, mode = "r")
ACON_file_2 = Nio.open_file(fn_2, mode = "r")
ACON_file_MCIP_2 = Nio.open_file(fn_MCIP_2, mode = "r")

In [4]:
df = Extract(fn, fn_2, fn_MCIP, fn_MCIP_2, species='O3')

var_spec:(20, 299, 459)
lat2d.shape:(299, 459)
lon2d.shape:(299, 459)
var2_spec:(4, 299, 459)
lat2d_2.shape:(299, 459)
lon2d_2.shape:(299, 459)
Total grid cells (for reference) = 137241
spec_CV.shape:(24, 299, 459)
Matching Latitude = True
Matching Longitude = True


In [5]:
df.arrayify()
df.countobs()
df.arraycreate()
df.datasetcreate()
df.csvcreate("Output_CSVs/20130715/O3_data_24hr_20130714.csv")

30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
30
<xarray.DataArray (dim_0: 30)>
array([0.01930427, 0.01914919, 0.01777671, 0.01699214, 0.01670269,
       0.0164493 , 0.01877624, 0.01738327, 0.01665987, 0.01637564,
       0.01627433, 0.01839085, 0.01712771, 0.01626303, 0.01618511,
       0.0159808 , 0.01922417, 0.01822515, 0.01700879, 0.01645576,
       0.01603238, 0.01968977, 0.01884574, 0.01763882, 0.01676479,
       0.01587426, 0.0193706 , 0.01803228, 0.0167462 , 0.01584166])
Dimensions without coordinates: dim_0
